<a href="https://colab.research.google.com/github/GermanM3/GermanM3/blob/master/Charlie's_Investment_Analysis_%26_Scoring_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pykrx finance-datareader ta

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import FinanceDataReader as fdr
from pykrx import stock

def analyze_charlies_method_with_scoring(ticker: str, years: int = 10, target_market_median_yield: float = 0.073):
    """
    찰리의 가치투자 5가지 판단 도구를 분석하고 점수화하는 함수

    Args:
        ticker (str): 종목 코드.
        years (int): ROE 및 수익률 분석 기간 (단위: 년).
        target_market_median_yield (float): 시장의 역사적 수익률 중위수.

    Returns:
        dict: 5가지 도구 분석 결과, 점수, 총점.
    """
    try:
        # 종목 코드와 기업명 가져오기
        stock_name = stock.get_market_ticker_name(ticker)

        end_date = datetime.today()
        start_date_fund_years = end_date.year - years

        # --- 펀더멘털 데이터 가져오기 (최적화) ---
        annual_fund_list = []
        for year in range(start_date_fund_years, end_date.year + 1):
            end_date_str_temp = f'{year}1231'
            start_date_str_temp = (datetime.strptime(end_date_str_temp, "%Y%m%d") - timedelta(days=15)).strftime("%Y%m%d")

            df_temp = stock.get_market_fundamental_by_date(start_date_str_temp, end_date_str_temp, ticker)
            if not df_temp.empty:
                annual_fund_list.append(df_temp.tail(1))

        if not annual_fund_list:
            return {"error": "펀더멘털 데이터가 충분하지 않습니다."}

        df_annual_fund = pd.concat(annual_fund_list)
        df_annual_fund.index = df_annual_fund.index.year

        # --- 가격 데이터 가져오기 ---
        start_date_price = (end_date - timedelta(days=365*years+30)).strftime("%Y%m%d")
        df_price = fdr.DataReader(ticker, start_date_price)

        if df_price.empty:
            return {"error": "주가 데이터를 가져오는 데 실패했습니다."}

        # === 2. 좋은 기업 선별 도구 분석 및 점수화 ===
        df_annual_fund['ROE'] = (df_annual_fund['EPS'] / df_annual_fund['BPS']) * 100
        historical_roe = df_annual_fund['ROE'].dropna().values

        if len(historical_roe) < 5:
            return {"error": "ROE 데이터가 충분하지 않습니다. (최소 5년)"}

        # ① ROE Median
        roe_median = np.median(historical_roe)
        roe_median_ok = roe_median >= 15
        roe_median_score = min(5, (roe_median / 15) * 5) + max(0, roe_median - 15)

        # ② ROE Adjusted CV
        q75 = np.percentile(historical_roe, 75)
        q25 = np.percentile(historical_roe, 25)
        roe_spread = q75 - q25
        roe_adjusted_cv = roe_median / roe_spread if roe_spread != 0 else np.inf
        roe_adjusted_cv_ok = roe_adjusted_cv >= 1
        roe_adjusted_cv_score = min(5, (roe_adjusted_cv / 1.0) * 5) + max(0, (roe_adjusted_cv - 1.0) * 5)

        # ③ Yield Median
        df_price['annual_return'] = df_price['Close'].pct_change(250)
        df_annual_returns = df_price.groupby(df_price.index.year).tail(1)['annual_return'].dropna()
        historical_yield_median = df_annual_returns.median()
        yield_median_ok = historical_yield_median > target_market_median_yield
        yield_median_score = max(0, (historical_yield_median - target_market_median_yield) * 100)

        # === 3. 좋은 주식 인식 도구 분석 및 점수화 ===

        # ④ PBR Location
        df_pbr_data = df_price.copy()
        df_pbr_data['year'] = df_pbr_data.index.year
        df_pbr_data = df_pbr_data.merge(df_annual_fund[['BPS']], left_on='year', right_index=True, how='left')
        df_pbr_data['BPS'] = df_pbr_data['BPS'].ffill()
        df_pbr_data['PBR'] = df_pbr_data['Close'] / df_pbr_data['BPS']
        historical_pbr = df_pbr_data['PBR'].dropna()

        if historical_pbr.empty:
            return {"error": "PBR 데이터를 계산할 수 없습니다."}

        current_pbr = historical_pbr.iloc[-1]
        pbr_location_rank = (historical_pbr.sort_values().to_list().index(current_pbr) / len(historical_pbr))
        pbr_location_ok = pbr_location_rank <= 0.25
        pbr_location_score = (1 - pbr_location_rank) * 10

        # ⑤ Value Trap
        df_price['EMA_10W'] = df_price['Close'].ewm(span=50, adjust=False).mean()
        df_price['EMA_40W'] = df_price['Close'].ewm(span=200, adjust=False).mean()
        current_value_trap_ratio = df_price['EMA_10W'].iloc[-1] / df_price['EMA_40W'].iloc[-1]
        value_trap_ok = current_value_trap_ratio >= 1
        value_trap_score = 10 if value_trap_ok else 0

        # === 최종 결과 취합 및 총점 계산 ===
        total_score = (roe_median_score + roe_adjusted_cv_score +
                       yield_median_score + pbr_location_score + value_trap_score)

        return {
            "종목코드": ticker,
            "종목명": stock_name,
            "ROE_Median": f"{roe_median:.2f}%",
            "ROE_Median_OK": roe_median_ok,
            "ROE_Median_Score": f"{roe_median_score:.2f}",
            "ROE_Adj_CV": f"{roe_adjusted_cv:.2f}",
            "ROE_Adj_CV_OK": roe_adjusted_cv_ok,
            "ROE_Adj_CV_Score": f"{roe_adjusted_cv_score:.2f}",
            "Yield_Median": f"{historical_yield_median*100:.2f}%",
            "Yield_Median_OK": yield_median_ok,
            "Yield_Median_Score": f"{yield_median_score:.2f}",
            "PBR_Location_Rank": f"{pbr_location_rank*100:.2f}%",
            "PBR_Location_OK": pbr_location_ok,
            "PBR_Location_Score": f"{pbr_location_score:.2f}",
            "Value_Trap_Ratio": f"{current_value_trap_ratio:.2f}",
            "Value_Trap_OK": value_trap_ok,
            "Value_Trap_Score": f"{value_trap_score:.2f}",
            "Total_Score": f"{total_score:.2f}",
            "총평": f"좋은 기업: {roe_median_ok and roe_adjusted_cv_ok and yield_median_ok}, "
                  f"좋은 주식: {pbr_location_ok and value_trap_ok}"
        }

    except Exception as e:
        return {"종목코드": ticker, "error": str(e)}

In [35]:
# === 예시 실행 ===
if __name__ == "__main__":
    sample_ticker = "138040"
    analysis_result = analyze_charlies_method_with_scoring(sample_ticker)

    if "error" in analysis_result:
        print(f"분석 실패: {analysis_result['error']}")
    else:
        print(f"[{analysis_result['종목코드']}] {analysis_result['종목명']} 찰리의 가치투자 분석 결과")
        for key, value in analysis_result.items():
            print(f"- {key}: {value}")


[138040] 메리츠금융지주 찰리의 가치투자 분석 결과
- 종목코드: 138040
- 종목명: 메리츠금융지주
- ROE_Median: 14.50%
- ROE_Median_OK: False
- ROE_Median_Score: 4.83
- ROE_Adj_CV: 2.42
- ROE_Adj_CV_OK: True
- ROE_Adj_CV_Score: 12.10
- Yield_Median: 19.18%
- Yield_Median_OK: True
- Yield_Median_Score: 11.88
- PBR_Location_Rank: 97.37%
- PBR_Location_OK: False
- PBR_Location_Score: 0.26
- Value_Trap_Ratio: 1.05
- Value_Trap_OK: True
- Value_Trap_Score: 10.00
- Total_Score: 39.08
- 총평: 좋은 기업: False, 좋은 주식: False
